In [ ]:
import pixiedust
from pyspark.sql import SparkSession

pixiedust.installPackage("org.apache.bahir:spark-sql-cloudant_2.11:0")
spark = SparkSession.builder.getOrCreate()

In [ ]:
def readDataFrameFromCloudant(host,user,pw,database):

    cloudantdata=spark.read.format("org.apache.bahir.cloudant"). \
    option("cloudant.host",host). \
    option("cloudant.username", user). \
    option("cloudant.password", pw). \
    load(database)

    cloudantdata.createOrReplaceTempView("washing")
    #spark.sql("SELECT * from washing").show()
    return cloudantdata

In [ ]:
#TODO Please provide your Cloudant credentials here
hostname = "pleasePutYourCredentialsHere"
user = "pleasePutYourCredentialsHere"
pw = "pleasePutYourCredentialsHere"
database = "washing" #as long as you didn't change this in the NodeRED flow the database name stays the same
cloudantdata=readDataFrameFromCloudant(hostname, user, pw, database)

In [ ]:
# You should see some data coming from the database
spark.sql("select * from washing").show()
# spark.sql("select avg(voltage), std(voltage), max(voltage) from washing").show()

In [ ]:
df = spark.sql("""
select mean(voltage) as mean, std(voltage) as std from (
    SELECT ROW_NUMBER() OVER (ORDER BY ts desc) AS row, * FROM washing where voltage is not null
) where
row < 30
""")
               
#df.show()
mean = df.first().mean
std = df.first().std
#spark.sql("SELECT * from (select abs( ({0}-voltage)/({1}+0.0001) ) as zscore, voltage,ROW_NUMBER() OVER (ORDER BY ts desc) AS row FROM washing where voltage is not null order by ts desc) where row < 20".format(mean,std)).show()
df = spark.sql("SELECT mean(zscore) as avgzscore from (select abs( ({0}-voltage)/({1}+0.0001) ) as zscore, voltage,ROW_NUMBER() OVER (ORDER BY ts desc) AS row FROM washing where voltage is not null order by ts desc) where row < 20".format(mean,std))
df.show()

In [ ]:
zscore = df.first().avgzscore

In [ ]:
import requests
if (zscore > 0.5):
    r = requests.get("https://1234thisismyname.mybluemix.net/edgemodelupdate?command=emergencyshutdown")
    r.status_code